In [46]:
from random import shuffle
import numpy as np
import re
import pandas as pd
import os 
from tqdm import tqdm
import pickle
import time

In [47]:
def shingle(text, k):
    """
    Generate shingles of length k from the given text.

    Args:
    - text (str): The input text.
    - k (int): The length of each shingle.

    Returns:
    - set: A set containing unique shingles extracted from the text.
    """
    shingle_set = []  
    # Iterate through the text to generate shingles
    for i in range(len(text) - k + 1):
        shingle_set.append(text[i:i + k])  
    return set(shingle_set) 


In [48]:
def binary_vector(vocabulary, text):
    """
    Create a binary vector representing the presence of shingles in the text.
    """
    binary_vector = np.zeros(len(vocabulary))  # Initialize a binary vector of zeros
    # Iterate through the vocabulary and check if each shingle is present in the text
    for i, shingle in enumerate(vocabulary):
        if shingle in text:
            binary_vector[i] = 1  # Set the corresponding index to 1 if the shingle is present
    return binary_vector 


In [49]:
def create_single_permutation(vocabulary):
    perm_vocab = list(range(1, len(vocabulary) + 1)) 
    shuffle(perm_vocab)  
    return perm_vocab 

def create_mperm(vocabulary_len, n_perm):
    """
        Create list containing multiple permutations of the vocabulary indices.
    """
    hashes = []
    # Generate n_perm permutations and append them to the list
    for _ in range(n_perm):
        hashes.append(create_single_permutation(vocabulary_len))
    return hashes 


In [50]:
def create_hash(text_vector: list, vocabulary: set, minhash_func: list):
    """
    Create a hash signature based on the text vector using minhash functions.

    """
    signature = []  # Initialize the hash signature
    for hash_func in minhash_func:  
        for i in range(1, len(vocabulary) + 1): 
            idx = hash_func.index(i)  
            signature_val = text_vector[idx]  
            if signature_val == 1:  
                signature.append(idx)  # Add the index to the hash signature
                break  # Move to the next hash function
    return signature  # Return the hash signature


In [51]:
def split_bands(signature, n_bands):
    """
    Split the signature matrix into bands.

    """
    
    # print signature with len == 0
    if len(signature) == 0:
        print("signature is empty")
        print(signature)
    assert len(signature) % n_bands == 0  # Ensure that the length of the signature is divisible by the number of bands
    n_rows = int(len(signature) / n_bands)  # Calculate the number of rows in each band
    bands = []
    # Split the signature matrix into bands
    for i in range(0, n_bands * n_rows, n_rows):
        bands.append(signature[i:i + n_rows])
    return bands 

def compare_bands(band1, band2):
    """
    Compare bands of two signature matrices.
    """
    # Compare corresponding bands
    for b1, b2 in zip(band1, band2):
        if b1 == b2:  # Check if the bands match
            return True 
    return False  # Return False if no matching band is found


In [52]:
def create_shingles_lists(words, k):
    """
    Create shingles lists for each address and update the vocabulary.

    Args:
    - addresses (array-like): A list of addresses or text data.
    - k (int): The length of each shingle.

    Returns:
    - set: The vocabulary containing unique shingles.
    - list: A list of shingles lists for each address.
    """
    # Initialize the vocabulary
    vocabulary = set()
    # Create a list to store shingles for each address
    shingles_lists = []

    # Update vocabulary and create shingles for each address
    for word in words:
        shingles = shingle(word, k)
        shingles_lists.append(list(shingles))  # Convert NumPy array to Python list
        vocabulary.update(shingles)

    return vocabulary, shingles_lists

In [53]:
# Function to store LSH signatures for all input values
def store_signatures_for_inputs(shingles_lists, k, vocabulary, minhash_func):
    for i, shingles_list in enumerate(tqdm(shingles_lists)):
        # Compute LSH for each address in the CSV
        shingles = shingles_list
        vector = binary_vector(vocabulary, shingles)
        signature = create_hash(vector, vocabulary, minhash_func)
        shingles_lists[i] = signature
    
    return shingles_lists

In [54]:
# Function to compare LSH signature of input text with stored signatures
def compare_with_stored_signatures(input_text, k, vocabulary, minhash_func,shingles_lists):
    input_shingles = shingle(input_text, k)
    input_vector = binary_vector(vocabulary, input_shingles)
    input_signature = create_hash(input_vector, vocabulary, minhash_func)
    
    similar_texts = []
    for i,shingles_sig in enumerate(shingles_lists):
        # len of signature  == 0 return index and signature and skip
        if len(shingles_sig) == 0:
            continue
        
        if compare_bands(split_bands(input_signature, 10), split_bands(shingles_sig, 10)):
            similar_texts.append(i)
    
    return similar_texts

In [55]:
# lower and special characters
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9]', ' ', text)
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text)
    return text


In [56]:
def find_similar_games(input_user_data, k, vocabulary, minhash_func, shingles_lists):
    """
    Find similar games based on the input user data.
    
    """
    # Preprocess the input address
    input_user_data = preprocess_text(input_user_data)

    # Compare with stored signatures
    similar_games = compare_with_stored_signatures(input_user_data, k, vocabulary, minhash_func, shingles_lists)
    return similar_games


In [62]:
df = pd.read_csv('steam-100k.csv')

#  Adding column names to the DataFrame
df.reset_index(drop=True, inplace=True)

df.head()


,id,game,purchase,hours,user,Pre_game
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0,the elder scrolls v skyrim
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0,the elder scrolls v skyrim
2,151603712,Fallout 4,purchase,1.0,0,fallout 4
3,151603712,Fallout 4,play,87.0,0,fallout 4
4,151603712,Spore,purchase,1.0,0,spore


In [63]:
# Preprocess the addresses on df new column names preprocessed_address
df['Pre_game'] = df['game'].apply(preprocess_text)
df.head()

,id,game,purchase,hours,user,Pre_game
0,151603712,The Elder Scrolls V Skyrim,purchase,1.0,0,the elder scrolls v skyrim
1,151603712,The Elder Scrolls V Skyrim,play,273.0,0,the elder scrolls v skyrim
2,151603712,Fallout 4,purchase,1.0,0,fallout 4
3,151603712,Fallout 4,play,87.0,0,fallout 4
4,151603712,Spore,purchase,1.0,0,spore


In [64]:
# Extract the preprocessed game names from the DataFrame
games = df['Pre_game'].values

# Display the first few preprocessed game names
games[:5]


array(['the elder scrolls v skyrim', 'the elder scrolls v skyrim',
       'fallout 4', 'fallout 4', 'spore'], dtype=object)

In [65]:
# Update the vocabulary and create shingles lists for the data
vocabulary, shingles_lists = create_shingles_lists(games, 2)
# Create permutations for the vocabulary indices
minhash_func = create_mperm((vocabulary), 50)
print(len(vocabulary))
print(shingles_lists[:2])

694
[['e ', 'sc', ' e', 'ky', 'sk', 'r ', 'ol', ' s', 'ro', 'ls', 'th', ' v', 'll', 'de', 's ', 'v ', 'im', 'he', 'er', 'ld', 'cr', 'yr', 'el', 'ri'], ['e ', 'sc', ' e', 'ky', 'sk', 'r ', 'ol', ' s', 'ro', 'ls', 'th', ' v', 'll', 'de', 's ', 'v ', 'im', 'he', 'er', 'ld', 'cr', 'yr', 'el', 'ri']]


In [66]:
# Store signatures for inputs
signatures = store_signatures_for_inputs(shingles_lists, 2, vocabulary, minhash_func)


  0%|          | 0/100000 [00:00<?, ?it/s]

100%|██████████| 100000/100000 [19:09<00:00, 87.01it/s]


In [67]:
# Save the shingles lists to a pickle file and also save the vocabulary and minhash functions
with open('signatures_meta.pkl', 'wb') as f:
    pickle.dump((vocabulary, minhash_func, signatures), f)
    print("LSH signatures saved to file.")


LSH signatures saved to file.


In [71]:
# Load the vocabulary, minhash functions, and shingles lists from the pickle file
with open('signatures_meta.pkl', 'rb') as f:
    vocabulary_store, minhash_func_store, signatures_store = pickle.load(f)
    print("LSH signatures loaded from file.")

LSH signatures loaded from file.


In [75]:

# Example usage
input_user_data = "Dota"
# Preprocess the input address
input_user_data = preprocess_text(input_user_data)
# Time the function
start_time = time.time()
# Find similar games
similar_games = find_similar_games(input_user_data, 2, vocabulary_store, minhash_func_store, signatures_store)
end_time = time.time()
# Length of similar games
total_similar_games = len(similar_games)
# Print the total number of similar games and the time taken
print(f"Total similar games found: {total_similar_games}")
print(f"Time taken: {end_time - start_time:} seconds")



Total similar games found: 4770
Time taken: 0.5688629150390625 seconds


In [76]:
# Coverting similar games to a DataFrame
similar_games_df = df.iloc[similar_games, :]
print(similar_games_df.shape)
similar_games_df

(4770, 6)


,id,game,purchase,hours,user,Pre_game
42,151603712,Dota 2,purchase,1.0,0,dota 2
43,151603712,Dota 2,play,0.5,0,dota 2
66,187131847,Dota 2,purchase,1.0,0,dota 2
67,187131847,Dota 2,play,2.3,0,dota 2
855,176410694,Dota 2,purchase,1.0,0,dota 2
...,...,...,...,...,...,...
99948,284555178,Dota 2,play,0.3,0,dota 2
99979,243532285,Dota 2,purchase,1.0,0,dota 2
99980,243532285,Dota 2,play,0.4,0,dota 2
99981,11731710,Dota 2,purchase,1.0,0,dota 2
